In [ ]:
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocoder = torch.hub.load(repo_or_dir='ex3ndr/supervoice-vocoder', model='bigvsan')
vocoder.to(device)
vocoder.eval()

flow = torch.hub.load(repo_or_dir='ex3ndr/supervoice-flow', model='flow', force_reload=True)
flow.to(device)
flow.eval()